# Local-Llama-Inference - GPU Detection and Analysis

Comprehensive guide to detecting GPU capabilities and optimizing configurations.

## GPU Information
- **Compute Capability**: Hardware generation (determines supported features)
- **VRAM**: Memory available for models and computation
- **UUID**: Unique identifier for tracking GPUs
- **Architecture**: NVIDIA architecture (sm_50, sm_61, sm_70, etc.)

In [ ]:
from local_llama_inference import detect_gpus, suggest_tensor_split, check_cuda_version
import subprocess
import platform

print("✅ Package imported")

## Step 1: System Information

In [ ]:
print("=" * 70)
print("SYSTEM INFORMATION")
print("=" * 70)

print(f"\n🖥️  Platform: {platform.system()} {platform.release()}")
print(f"   Processor: {platform.processor()}")
print(f"   Python Version: {platform.python_version()}")

# Check CUDA
try:
    cuda_version = check_cuda_version()
    print(f"\n🎮 CUDA Setup:")
    if cuda_version:
        print(f"   ✅ CUDA Available: {cuda_version}")
    else:
        print(f"   ⚠️  CUDA not found")
except Exception as e:
    print(f"   Error checking CUDA: {e}")

## Step 2: Detect GPUs

In [ ]:
print("=" * 70)
print("GPU DETECTION")
print("=" * 70)

gpus = detect_gpus()

if not gpus:
    print("\n⚠️  No GPUs detected!")
    print("   Make sure you have:")
    print("   - NVIDIA GPU installed")
    print("   - NVIDIA drivers installed")
    print("   - CUDA toolkit installed")
else:
    print(f"\n✅ Detected {len(gpus)} GPU(s):\n")
    for i, gpu in enumerate(gpus):
        print(f"GPU {i}: {gpu['name']}")
        print(f"  Compute Capability: {gpu['compute_capability']}")
        print(f"  VRAM: {gpu['memory_mb']} MB ({gpu['memory_mb']/1024:.2f} GB)")
        print(f"  UUID: {gpu['uuid']}")
        print()

## Step 3: GPU Architecture Analysis

In [ ]:
print("=" * 70)
print("GPU ARCHITECTURE ANALYSIS")
print("=" * 70)

# NVIDIA GPU Architecture reference
architectures = {
    (3, 0): ("Kepler", "K80, K40"),
    (5, 0): ("Maxwell", "GTX 750, GTX 950"),
    (6, 1): ("Pascal", "GTX 1060, GTX 1080"),
    (7, 0): ("Volta", "Tesla V100"),
    (7, 5): ("Turing", "RTX 2060, RTX 2080"),
    (8, 0): ("Ampere", "RTX 3060, RTX 3090"),
    (8, 6): ("Ada", "RTX 4080, RTX 6000"),
    (8, 9): ("Hopper", "H100, H200"),
}

if gpus:
    for i, gpu in enumerate(gpus):
        cc = gpu['compute_capability']
        major, minor = cc.split('.')
        major, minor = int(major), int(minor)
        
        print(f"\nGPU {i}: {gpu['name']}")
        print(f"  Compute Capability: {cc}")
        
        # Find architecture
        arch_name = None
        arch_examples = None
        for (maj, min), (name, examples) in architectures.items():
            if major == maj:
                arch_name = name
                arch_examples = examples
                break
        
        if arch_name:
            print(f"  Architecture: {arch_name}")
            print(f"  Examples: {arch_examples}")
        else:
            print(f"  Architecture: Unknown")
        
        # Feature support
        print(f"\n  Feature Support:")
        print(f"    ✅ CUDA Compute: Yes")
        print(f"    ✅ Float32: Yes")
        print(f"    {'✅' if major >= 5 else '❌'} Float64 (if sm_50+): Yes")
        print(f"    {'✅' if major >= 7 else '❌'} Tensor Cores (if sm_70+): Yes")
        print(f"    {'✅' if major >= 8 else '❌'} Structured Sparsity (if sm_80+): Yes")
else:
    print("No GPUs detected")

## Step 4: VRAM Analysis

In [ ]:
print("=" * 70)
print("VRAM ANALYSIS")
print("=" * 70)

if gpus:
    total_vram = sum(gpu['memory_mb'] for gpu in gpus)
    
    print(f"\nTotal VRAM: {total_vram} MB ({total_vram/1024:.2f} GB)")
    
    print(f"\nPer-GPU VRAM:")
    for i, gpu in enumerate(gpus):
        vram_gb = gpu['memory_mb'] / 1024
        pct = (gpu['memory_mb'] / total_vram * 100) if total_vram > 0 else 0
        print(f"  GPU {i}: {vram_gb:.2f} GB ({pct:.1f}%)")
    
    # Model capacity estimates
    print(f"\n📊 Model Capacity (Rough Estimates):")
    print(f"\n  For {total_vram/1024:.2f} GB total VRAM:")
    
    model_sizes = [
        ("Phi-2 Q4 (1.4 GB)", 1.4),
        ("Mistral 7B Q4 (4.3 GB)", 4.3),
        ("Llama 2 7B Q4 (3.8 GB)", 3.8),
        ("Llama 2 13B Q4 (7.3 GB)", 7.3),
    ]
    
    for model_name, size_gb in model_sizes:
        fits = "✅ Yes" if size_gb <= total_vram/1024 * 0.8 else "❌ No (tight fit)"
        print(f"    {model_name}: {fits}")
else:
    print("No GPUs detected")

## Step 5: Tensor Split Recommendation

In [ ]:
print("=" * 70)
print("TENSOR SPLIT RECOMMENDATION")
print("=" * 70)

if gpus:
    tensor_split = suggest_tensor_split(gpus)
    
    print(f"\n✅ Recommended Tensor Split for Multi-GPU:\n")
    
    if isinstance(tensor_split, list):
        total = sum(tensor_split)
        for i, split in enumerate(tensor_split):
            pct = split * 100
            print(f"  GPU {i}: {pct:.1f}%")
        
        print(f"\n📝 Explanation:")
        print(f"  - Each GPU receives a fraction of model layers")
        print(f"  - Proportional to VRAM size")
        print(f"  - Requires NCCL for GPU communication")
        print(f"  - Use with: LlamaServer(..., tensor_split={tensor_split})")
    else:
        print(f"  Single GPU - No tensor split needed")
        print(f"\n📝 Explanation:")
        print(f"  - Only 1 GPU detected")
        print(f"  - All model layers fit on one GPU")
else:
    print("No GPUs detected")

## Step 6: GPU Layer Distribution Calculator

In [ ]:
print("=" * 70)
print("GPU LAYER DISTRIBUTION CALCULATOR")
print("=" * 70)

if gpus:
    # Example model: 32 layers
    total_layers = 32
    total_vram = sum(gpu['memory_mb'] for gpu in gpus)
    
    print(f"\nExample: Model with {total_layers} layers\n")
    
    if isinstance(suggest_tensor_split(gpus), list):
        tensor_split = suggest_tensor_split(gpus)
        print(f"Layer Distribution Across {len(gpus)} GPUs:\n")
        
        for i, split in enumerate(tensor_split):
            layers = int(total_layers * split)
            print(f"  GPU {i}: {layers} layers ({split*100:.1f}% of model)")
    else:
        print(f"Layer Distribution (Single GPU):\n")
        print(f"  GPU 0: {total_layers} layers (100% of model)")
else:
    print("No GPUs detected")

## Step 7: GPU Utilization with nvidia-smi

In [ ]:
print("=" * 70)
print("REAL-TIME GPU STATUS (nvidia-smi)")
print("=" * 70)

try:
    print("\n")
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free,temperature.gpu,utilization.gpu,utilization.memory',
         '--format=csv,noheader'],
        capture_output=True,
        text=True,
        timeout=5
    )
    
    if result.returncode == 0:
        lines = result.stdout.strip().split('\n')
        for line in lines:
            parts = [p.strip() for p in line.split(',')]
            if len(parts) >= 9:
                idx, name, driver, total_mem, used_mem, free_mem, temp, gpu_util, mem_util = parts[:9]
                print(f"GPU {idx}: {name}")
                print(f"  Driver: {driver}")
                print(f"  Memory: {used_mem} / {total_mem} (Free: {free_mem})")
                print(f"  Temperature: {temp}")
                print(f"  GPU Utilization: {gpu_util}")
                print(f"  Memory Utilization: {mem_util}")
                print()
    else:
        print(f"Error running nvidia-smi: {result.stderr}")
except subprocess.TimeoutExpired:
    print("nvidia-smi timed out")
except FileNotFoundError:
    print("nvidia-smi not found - install NVIDIA drivers")
except Exception as e:
    print(f"Error: {e}")
    print("\nTip: Run 'nvidia-smi' in terminal to check GPU status")

## Step 8: Configuration Recommendations

In [ ]:
print("=" * 70)
print("CONFIGURATION RECOMMENDATIONS")
print("=" * 70)

if gpus:
    print(f"\n📋 Recommended LlamaServer Configuration:\n")
    
    total_vram = sum(gpu['memory_mb'] for gpu in gpus)
    total_vram_gb = total_vram / 1024
    
    # Determine n_ctx based on VRAM
    if total_vram_gb < 4:
        n_ctx = 1024
        n_gpu_layers = 20
    elif total_vram_gb < 8:
        n_ctx = 2048
        n_gpu_layers = 33
    else:
        n_ctx = 4096
        n_gpu_layers = 40
    
    tensor_split = suggest_tensor_split(gpus)
    
    print(f"server = LlamaServer(")
    print(f"    model_path=\"path/to/model.gguf\",")
    print(f"    n_gpu_layers={n_gpu_layers},  # GPU layers (higher = more GPU)")
    print(f"    n_ctx={n_ctx},  # Context size (based on {total_vram_gb:.1f} GB VRAM)")
    print(f"    n_threads=4,  # CPU threads")
    
    if isinstance(tensor_split, list) and len(tensor_split) > 1:
        print(f"    tensor_split=[{', '.join(f'{s:.2f}' for s in tensor_split)}],  # Multi-GPU")
    
    print(f")")
    
    print(f"\n💡 Tips:")
    print(f"  • n_gpu_layers=33 means offload 33 layers to GPU")
    print(f"  • Higher n_gpu_layers = faster but uses more VRAM")
    print(f"  • Adjust n_ctx if you run out of memory")
    print(f"  • For multi-GPU, tensor_split distributes layers")
else:
    print("No GPUs detected - CPU-only mode")

## Step 9: Troubleshooting

In [ ]:
print("=" * 70)
print("TROUBLESHOOTING GUIDE")
print("=" * 70)

print(f"\n❌ Issue: No GPUs detected")
print(f"\nSolutions:")
print(f"  1. Check GPU installed: lspci | grep NVIDIA")
print(f"  2. Install NVIDIA drivers: nvidia-driver package")
print(f"  3. Install CUDA: Download from nvidia.com")
print(f"  4. Verify: Run 'nvidia-smi'")

print(f"\n❌ Issue: Out of Memory (OOM) errors")
print(f"\nSolutions:")
print(f"  1. Reduce n_gpu_layers (offload fewer layers)")
print(f"  2. Reduce n_ctx (smaller context)")
print(f"  3. Use smaller model (fewer parameters)")
print(f"  4. Enable tensor split for multi-GPU")

print(f"\n❌ Issue: Slow inference")
print(f"\nSolutions:")
print(f"  1. Increase n_gpu_layers (offload more to GPU)")
print(f"  2. Check GPU utilization: nvidia-smi")
print(f"  3. Use larger batch size if applicable")
print(f"  4. Add more GPUs for tensor parallelism")

print(f"\n❌ Issue: NCCL errors (multi-GPU)")
print(f"\nSolutions:")
print(f"  1. Ensure all GPUs are from same vendor (NVIDIA)")
print(f"  2. Check NCCL is installed: pip show nccl")
print(f"  3. Verify tensor_split format is correct")
print(f"  4. Use nvidia-smi to check GPU connectivity")

print(f"\n💬 Getting Help:")
print(f"  • GitHub Issues: https://github.com/Local-Llama-Inference/...")
print(f"  • Documentation: Check README.md")
print(f"  • Run: llama-inference info  (for debugging)")

## Key Takeaways

1. **GPU Detection**: Use `detect_gpus()` to get GPU info
2. **Compute Capability**: Determines feature support (sm_50+)
3. **VRAM**: Limits model size and context length
4. **Tensor Split**: Distributes models across multiple GPUs
5. **Optimization**: Balance GPU layers, context size, and batch size
6. **Monitoring**: Use `nvidia-smi` to check utilization

## Next Steps

- Run `01_quick_start.ipynb` to start generating text
- Try different models and configurations
- Monitor GPU usage during inference
- Optimize parameters for your use case

## Useful Commands

```bash
# Check GPU
nvidia-smi

# Real-time GPU monitoring
nvidia-smi dmon

# Check CUDA version
nvcc --version

# Package info
llama-inference info
```